# 02-01 - Was sind Embeddings

In diesem Lab werden wir untersuchen, wie wir unsere eigenen Daten in die von Azure OpenAI verwendeten Modelle einbringen können.

Wir beginnen wie üblich damit, eine Verbindung zum Azure OpenAI-Dienst herzustellen.

**HINWEIS**: Wie in den vorherigen Labs verwenden wir die Werte aus der Datei `.env`, die sich im Hauptverzeichnis dieses Repositorys befindet.

In [2]:
import os
from langchain.llms import AzureOpenAI
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage
from dotenv import load_dotenv

# Laden der Umgebungsvariablen
if load_dotenv():
    print("Azure OpenAI-Endpunkt gefunden: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Keine .env-Datei gefunden")

# Erstellen einer Instanz von Azure OpenAI
llm = AzureChatOpenAI(
    azure_deployment = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")
)

Azure OpenAI-Endpunkt gefunden: https://azure-openai-ibit-hackathon.openai.azure.com/


Lassen Sie uns damit beginnen, der KI eine einfache Frage zu stellen.

In [3]:
r = llm.invoke("Tell me about the latest Deadpool movie. When was it released? What is it about?")

# Ausgabe der Antwort
print(r.content)

As of my last knowledge update in October 2023, **"Deadpool 3"** is the latest installment in the *Deadpool* film series. It is highly anticipated and has generated significant buzz among fans, particularly because it marks the character's full integration into the Marvel Cinematic Universe (MCU) after Disney's acquisition of 21st Century Fox.

### Release Date
*Deadpool 3* is currently scheduled to be released in **May 2024**, after experiencing delays due to the industry-wide WGA and SAG-AFTRA strikes. This will make it the first *Deadpool* film released under Marvel Studios and the Disney banner.

### Plot Details
While Marvel has remained tight-lipped about specific story details, here's what is known or heavily rumored about *Deadpool 3* as of October 2023:
- The film will continue Ryan Reynolds’ portrayal of the wisecracking, fourth-wall-breaking antihero Wade Wilson/Deadpool.
- **Hugh Jackman returns as Wolverine**, reprising his iconic role from the *X-Men* franchise. Reports i

Was fällt Ihnen an der Antwort auf?

Der neueste "Deadpool"-Film heisst "Deadpool and Wolverine". Abhängig vom Modell und der Version, die Sie verwenden, kann es sein, dass das Modell einen der vorherigen Filme als den neuesten bezeichnet oder dass es zwar weiß, dass es einen neuen Film gibt, aber denkt, dass dieser noch nicht veröffentlicht wurde.

OpenAI-Modelle werden auf einer großen Menge Daten trainiert, aber dies geschieht zu einem bestimmten Zeitpunkt, der vom jeweiligen Modell abhängt. Daher haben viele Modelle keine Informationen über Ereignisse, die in sehr aktuellen Monaten oder Jahren stattgefunden haben.

Um der KI zu helfen, können wir ihr zusätzliche Informationen bereitstellen. Das gleiche Vorgehen würden Sie anwenden, wenn Sie möchten, dass die KI mit Ihren firmeneigenen Daten arbeitet. Die KI weiß nichts über Informationen, die nicht öffentlich zugänglich sind. Wenn Sie also möchten, dass die KI mit diesen Informationen arbeitet, müssen Sie diese Informationen in das Modell einbringen.

Das Problem ist, dass Sie das Modell dazu nicht wirklich neu trainieren können. Die Modelle sind vorab trainiert, und ein erneutes Training ist aufwendig und zeitintensiv.

Es gibt jedoch Möglichkeiten, KI-Modelle trotzdem mit neuen Daten arbeiten zu lassen. Die beliebteste Methode ist die Verwendung von *Embeddings* (Einbettungen), die wir in den nächsten Abschnitten untersuchen werden.

## Eigene Daten einbringen

Langchain bietet eine Reihe nützlicher Tools, darunter solche, die den Umgang mit externen Dokumenten vereinfachen. Im folgenden Beispiel verwenden wir den `DirectoryLoader`, der mehrere Dateien aus einem Verzeichnis lesen kann, und den `UnstructuredMarkdownLoader`, der Dateien im Markdown-Format verarbeiten kann. Wir nutzen diese, um eine Reihe von Markdown-Dateien zu verarbeiten, die Details über kürzlich veröffentlichte Filme enthalten.

In [5]:
from langchain.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader

data_dir = "movies"

documents = DirectoryLoader(path=data_dir, glob="*.md", show_progress=True, loader_cls=UnstructuredMarkdownLoader).load()

  0%|          | 0/17 [00:00<?, ?it/s]

100%|██████████| 17/17 [00:02<00:00,  5.81it/s]


Wir haben nun ein Objekt `documents`, das alle Informationen aus unseren Markdown-Dokumenten über Filme enthält.

Wir können die `question_answering`-Chain verwenden, um der KI Zugriff auf unsere Dokumente zu geben und dann dieselbe Frage zu den Deadpool-Filmen erneut stellen.

In [6]:
# Frage-Antwort-Chain
from langchain.chains.question_answering import load_qa_chain

# Vorbereiten der Chain und der Abfrage
chain = load_qa_chain(llm)
query = "Tell me about the latest Deadpool movie. When was it released? What is it about?"

result = chain.invoke({'input_documents': documents, 'question': query})

print (result['output_text'])

/tmp/ipykernel_252410/829329869.py:5: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm)


The latest *Deadpool* movie, titled **"Deadpool & Wolverine"**, is an action-packed, comedy-heavy entry in the superhero genre. It is set to be released on **July 24, 2024**.

### Plot Overview:
The movie follows Wade Wilson, aka Deadpool, who has left his mercenary days behind and leads a listless civilian life. However, when his homeworld faces an existential threat, he is reluctantly forced to suit up again. Teaming up with a very reluctant Wolverine (played by Hugh Jackman), Deadpool must confront a new enemy while navigating a chaotic multiverse. Along the way, the film delivers plenty of humor, epic action sequences, and nods to both characters' origins and histories.

### Reception:
Early reviews suggest that while the plot might not be the strongest aspect of the film, the comedy, action, and chemistry between Ryan Reynolds (Deadpool) and Hugh Jackman (Wolverine) are highly praised. Fans have lauded the film’s humor, nostalgic throwbacks, and the playful jabs at the superhero g

Super! Das Modell kennt nun die richtigen Details zum neuesten Deadpool-Film.

Allerdings gibt es etwas im Hintergrund zu beachten! Werfen wir einen Blick darauf, was hinter den Kulissen passiert.

Wir werden zwei Dinge tun. Erstens werden wir den Parameter `verbose=True` zur Chain hinzufügen, und zweitens werden wir die Ausführung der Chain in einen Callback einbetten, um die Anzahl der verbrauchten Tokens zu erfassen.

In [7]:
# Unterstützung für Callbacks
from langchain.callbacks import get_openai_callback

# Vorbereiten der Chain und der Abfrage
chain = load_qa_chain(llm, verbose=True)
query = "Tell me about the latest Deadpool movie. When was it released? What is it about?"

# Ausführen der Chain unter Verwendung des Callbacks, um die Anzahl der verwendeten Tokens zu erfassen
with get_openai_callback() as callback:
    chain.invoke({'input_documents': documents, 'question': query})
    total_tokens = callback.total_tokens

print(f"Gesamtanzahl verwendeter Tokens: {total_tokens}")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
iNumber Number: Jozi Gold

Overview

When an undercover cop is tasked with investigating a historic gold heist in Johannesburg, he’s forced to choose between his conscience and the law.

Details

Release Date: 2023-06-23

Genres: Crime, Action, Thriller

Popularity: 719.085

Vote Average: 6.3

Keywords:

Extraction 2

Overview

Tasked with extracting a family who is at the mercy of a Georgian gangster, Tyler Rake infiltrates one of the world's deadliest prisons in order to save them. But when the extraction gets hot, and the gangster dies in the heat of battle, his equally ruthless brother tracks down Rake and his team to Vienna, in order to get revenge.

Details

Release Date: 2023-06-09

Genres: Ac

In der Ausgabe des letzten Codeabschnitts sollten Sie viele Informationen sehen. Am Ende sollte die Anzahl der verwendeten Tokens angezeigt werden. Sie werden möglicherweise überrascht sein, dass die Abfrage je nach verwendetem Modell zwischen 5000 und 6000 Tokens in Anspruch nehmen kann. Das ist eine Menge!

Mit der Option `verbose=True` wird in der restlichen Ausgabe auch das Prompt angezeigt, das für die Abfrage zusammengestellt wurde. Wenn Sie durch die Ausgabe scrollen, werden Sie feststellen, dass das Prompt **alle** Informationen aus unseren Dokumenten enthält – daher werden so viele Tokens verbraucht.

Wie wir bereits besprochen haben, haben KI-Modelle eine maximale Anzahl von Tokens, die verwendet werden können, und ein Preismodell, das auf der Anzahl der verbrauchten Tokens basiert. In diesem Beispiel sind die Dokumente relativ klein und es gibt nur 20 davon, aber wenn wir mit größeren Dokumenten und mehr davon arbeiten wollten, würde diese Methode schnell kostspielig werden und schließlich die Token-Grenze erreichen.

## Embeddings

Die Lösung für den Umgang mit großen Datenmengen aus externen Quellen besteht in der Verwendung von *Embeddings*. OpenAI stellt Embedding-Modelle zur Verfügung, mit denen menschenlesbare Informationen nach Bedeutung und Intention analysiert werden können. Die Ausgabe eines Embedding-Modells sind numerische Daten, sogenannte *Vektoren*. Diese ermöglichen es Computern, ähnliche Informationen zu gruppieren. Die Vektoren werden dann in einem *Vektorstore* gespeichert. Wenn Sie eine Frage stellen möchten, wird der Abfragetext erneut mithilfe eines Embedding-Modells in Vektoren umgewandelt. Diese Vektor-Daten, die Ihre Abfrage repräsentieren, können dann in der Datenbank durchsucht werden. Die gefundenen ähnlichen Vektoren sind vermutlich eine gute Antwort auf Ihre Frage.

Um zu verhindern, dass ein Prompt mit einer großen Anzahl an Tokens überlastet wird, senden wir nicht mehr alle unsere Dokumente an die KI, sondern führen zuerst eine Vektorsuche durch, um eine relevante Teilmenge der Dokumente zu finden. Nur diese relevante Teilmenge wird dann in das Prompt eingebunden.

Beginnen wir mit der Initialisierung einer Instanz eines Embedding-Modells. Sie werden feststellen, dass dies ähnlich ist, wie wenn wir eine unserer Modelldepoyments für Anfragen initialisieren, nur dass wir in diesem Fall ein Embedding-Modell angeben. Typischerweise wurde hierfür `text-embedding-ada-002` verwendet, aber inzwischen gibt es neuere Alternativen.

In [8]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings_model = AzureOpenAIEmbeddings(    
    azure_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
    openai_api_version = os.getenv("OPENAI_EMBEDDING_API_VERSION"),
    model= os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")
)

Jetzt, wo wir ein Modell zur Erstellung von Embeddings initialisiert haben, können wir beginnen, unsere Dokumente einzubetten.

Wie im vorherigen Beispiel verwenden wir den in Langchain integrierten Loader, um die Dokumente aus einem Verzeichnis zu laden.

In [9]:
documents = DirectoryLoader(path=data_dir, glob="*.md", show_progress=True, loader_cls=UnstructuredMarkdownLoader).load()

100%|██████████| 17/17 [00:00<00:00, 124.78it/s]


Der nächste Schritt ist die Verwendung eines *Splitters*. Ein Splitter ermöglicht es uns, längere Dokumente in kleinere Teile zu zerlegen, damit wir nicht riskieren, das Token-Limit zu überschreiten, wenn wir unsere Daten an das Embedding-Modell senden.

In [10]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
document_chunks = text_splitter.split_documents(documents)

Created a chunk of size 2658, which is longer than the specified 1000
Created a chunk of size 2255, which is longer than the specified 1000


Im nächsten Schritt werden die aufgeteilten Dokumente in Vektoren konvertiert, indem wir die Daten durch ein Embedding-Modell schicken. Die resultierenden Vektoren werden in einer Vektor-Datenbank gespeichert. In diesem Beispiel verwenden wir die Datenbank **Qdrant** (ausgesprochen „Quadrant“). Wir initialisieren sie mit der Option `location=":memory:"`, damit die Datenbank im Speicher und nicht auf der Festplatte gespeichert wird.

In [11]:
from langchain.vectorstores import Qdrant

qdrant = Qdrant.from_documents(
    document_chunks,
    embeddings_model,
    location=":memory:",
    collection_name="movies",
)

Der obige Codeabschnitt kümmert sich um die Initialisierung der Qdrant-Datenbank, das Durchführen der Dokumente durch das Embedding-Modell und das Speichern der resultierenden Vektoren in der Datenbank.

Als Nächstes definieren wir einen *Retriever*. In Langchain ist ein Retriever ein Interface, das Suchergebnisse aus dem Vektorspeicher zurückgibt. Wir erstellen also einen Retriever für unsere Qdrant-Datenbank.

In [12]:
retriever = qdrant.as_retriever()

Als Nächstes definieren wir eine `RetrievalQA`-Chain. Diese kümmert sich um den Prozess, eine Frage zu beantworten, indem zuerst in der Vektor-Datenbank gesucht und anschließend die Ergebnisse an unser KI-Modell übergeben werden.

In [13]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

Jetzt führen wir unsere Abfrage erneut aus – mit einer kleinen Änderung.

Möglicherweise denken Sie: „Es ist nicht überraschend, dass die KI jetzt über den neuesten Deadpool-Film Bescheid weiß, schließlich haben wir sie darüber informiert!“ Also versuchen wir zu zeigen, dass die KI tatsächlich ein wenig nachdenken und Schlüsse ziehen kann.

Falls Sie kein Fan dieser Filme sind: Deadpool entstammt den Marvel-Comics. Alle Filme, die aus Marvel-Comics hervorgehen, werden üblicherweise als Teil des Marvel Cinematic Universe betrachtet (oft MCU genannt). Wir haben in unseren Daten weder Marvel noch MCU erwähnt. Wenn wir die Frage also ein wenig anpassen und nach dem MCU statt nach Deadpool fragen, sollte die KI durch Nachdenken erkennen, was gemeint ist.

In [14]:
query = "Tell me about the latest MCU movie. When was it released? What is it about?"
result = qa.invoke(query)
print(result['result'])

The latest Marvel Cinematic Universe-related film is **"Deadpool & Wolverine"**, which was released on **July 24, 2024**. It's not directly part of the main MCU timeline but ties into the shared multiverse concept, blending the humor and edginess of Deadpool with the gritty nature of Wolverine.

### Plot:
The film follows a seemingly listless **Wade Wilson (Deadpool)** living a mundane civilian life, having left behind his days as a morally flexible mercenary. However, when his world faces an existential threat, he reluctantly suits up again and teams up with an equally reluctant **Logan (Wolverine)**. The story is packed with action, comedy, meta-humor, fourth-wall-breaking moments, and nostalgic references to past Fox/Marvel films, delivering a funny and chaotic ride.

It plays with familiar superhero tropes while also addressing the disorganized state of the multiverse, something Deadpool humorously critiques. Fans also loved the array of cameos and fight scenes, highlighting its co

Wenn alles geklappt hat, sollte die KI geantwortet haben, dass der neueste MCU-Film "Deadpool & Wolverine" ist, der im Juli 2024 veröffentlicht wurde.

Wir bekommen also die erwartete Antwort, doch schauen wir uns nun den Hauptgrund an, weshalb wir diesen ganzen Aufwand betreiben. Wurde die Anzahl der verwendeten Tokens verringert? Nutzen wir wieder unsere Callback-Technik, um das herauszufinden.

In [15]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as callback:
    qa.invoke(query)
    total_tokens = callback.total_tokens

print(f"Gesamtanzahl verwendeter Tokens: {total_tokens}")

Gesamtanzahl verwendeter Tokens: 1037


Die genaue Anzahl der Tokens kann variieren, aber es sollte offensichtlich sein, dass bei dieser Abfrage nun deutlich weniger Tokens verbraucht werden, oft um rund 2.000 weniger.

KI-Orchestrierungsbibliotheken wie Langchain und Semantic Kernel können den Prozess des Einbettens, Vektorisierens und Suchens stark vereinfachen. Im vorherigen Abschnitt haben wir den Prozess des Dokument-Splittings, Einbettens, Vektorisierens, Speicherns von Vektoren in einer Datenbank und Erstellen eines Retrievers Schritt für Schritt durchlaufen. Im nächsten Abschnitt verwenden wir erneut den `DirectoryLoader` von Langchain, um unsere Markdown-Dokumente zu laden und zu verarbeiten. Diesmal nutzen wir jedoch einen `VectorstoreIndexCreator`. Sie werden sehen, dass nur wenige Parameter nötig sind – das zu verwendende Embedding-Modell und die Quelldaten (`loader`). Hinter den Kulissen führt der `VectorstoreIndexCreator` allerdings sämtliche Schritte aus, die wir manuell in den vorherigen Abschnitten ausgeführt haben.

In [16]:
from langchain.indexes import VectorstoreIndexCreator

loader = DirectoryLoader(path=data_dir, glob="*.md", show_progress=True, loader_cls=UnstructuredMarkdownLoader)

index = VectorstoreIndexCreator(
    embedding=embeddings_model
    ).from_loaders([loader])

/home/yimiwang/miniconda3/envs/ibit-hackathon/lib/python3.13/site-packages/langchain/indexes/vectorstore.py:171: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(
100%|██████████| 17/17 [00:00<00:00, 143.75it/s]


Um nun eine Abfrage gegen unsere Daten zu starten, müssen wir nur das Prompt angeben und dann den oben erstellten Index aufrufen, dem wir das Modell (`llm`) und die Frage übergeben.

In [17]:
query = "Tell me about the latest Deadpool movie. When was it released? What is it about?"
index.query(llm=llm, question=query)

'The latest Deadpool movie, titled *Deadpool & Wolverine*, was released on **July 24, 2024**. It combines the irreverent humor of Deadpool with Wolverine\'s gritty persona, creating a unique superhero team-up.\n\nThe story follows Wade Wilson (Deadpool), who has been living a quiet civilian life after giving up his morally flexible mercenary ways. However, when his homeworld faces an existential threat, he reluctantly takes up the mantle of Deadpool once again. Adding to the mix is Wolverine, who is also reluctant to join forces. Together, they must confront the hilariously menacing villain "Paradox" (played by Matthew Macfadyen) and the powerful "Cassandra Nova" (Emma Corrin), Xavier\'s long-estranged twin sister, to save their timeline from erasure.\n\nWhile the plot has been criticized for being a bit disorganized and lacking strong villains, the film is praised for its laugh-out-loud humor, creative storytelling, amazing cameos, and fight scenes. It also features plenty of referenc

Sie sehen, dies ist eine sehr einfache Möglichkeit, Embeddings und Vektoren in einer KI-Anwendung einzusetzen und eignet sich hervorragend, um schnell zu starten.

Wir können erneut die Callback-Methode nutzen, um sicherzustellen, dass weiterhin eine geringere Anzahl an Tokens verbraucht wird.

In [18]:
# Ausführen der Chain, um die Anzahl der verwendeten Tokens zu erfassen
with get_openai_callback() as callback:
    index.query(llm=llm, question=query)
    total_tokens = callback.total_tokens

print(f"Gesamtanzahl verwendeter Tokens: {total_tokens}")

Gesamtanzahl verwendeter Tokens: 1069
